# Particle Swarm Optimization

## Implementation of the PSO optimizer


In [1]:
# Some libraries
from scipy import *
from math import *
from matplotlib.pyplot import *
from functools import *
import sys
from tqdm import tqdm
import time

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [8]:
class PSO:
    """Particle Swarm Optimizer"""
    
    def __init__(self, eval_func,
                 dim=2, 
                 inf=0, 
                 sup=100, 
                 n_agents=30, 
                 n_cycles=500, 
                 PSI=0.9, 
                 CMAX=0.8,
                 isbetter_func=lambda f1, f2: f1 < f2,
                 bounding_func=None,
                 init_func=None,
                 log_func=None):
        self.dim = dim
        self.inf = inf
        self.sup = sup
        self.n_agents = n_agents
        self.n_cycles = n_cycles
        self.PSI = PSI
        self.CMAX = CMAX
        
        self.eval_func = eval_func
        self.isbetter_func = isbetter_func
        self.bounding_func = bounding_func if bounding_func is not None else self.bounding
        self.init_func = init_func if init_func is not None else self.plain_init
        self.log_func = log_func if log_func is not None else self.display_result
        
        self.population = []
        self.best_agent = None
        
    def init_population(self):
        positions = self.init_func()
        fits = [self.eval_func(pos) for pos in positions]
        self.population = [{'vit':[0]*self.dim, 'pos':positions[i], 'fit':fits[i], 'bestpos':positions[i], 'bestfit':fits[i], 'bestvois':[]} for i in range(self.n_agents)]
        return self.population
    
    def max_particle(self, p1, p2):
        if self.isbetter_func(p1["fit"], p2["fit"]):
            return p1 
        else:
            return p2
    
    def get_best_agent(self):
        return dict(reduce(lambda acc, e: self.max_particle(acc,e),self.population[1:],self.population[0]))
    
    def update_agent(self, particle):
        nv = dict(particle)
        if self.isbetter_func(particle["fit"],particle["bestfit"]):
            nv['bestpos'] = particle["pos"][:]
            nv['bestfit'] = particle["fit"]
        nv['bestvois'] = self.best_agent["bestpos"][:]
        return nv

    def move_agent(self, particle):
        nv = dict(particle)

        velocity = [0]*self.dim
        for i in range(self.dim):
            velocity[i] = (particle["vit"][i]*self.PSI + \
            self.CMAX*random.uniform()*(particle["bestpos"][i] - particle["pos"][i]) + \
            self.CMAX*random.uniform()*(particle["bestvois"][i] - particle["pos"][i]))

        new_pos = [particle["pos"][i] + velocity[i] for i in range(self.dim)]
        try:
            position = self.bounding_func(new_pos)   
        except ValueError:
            position = particle["pos"]

        nv['vit'] = velocity
        nv['pos'] = position
        nv['fit'] = self.eval_func(position)
        return nv
    
    def bounding(self, position):
        return [min(self.sup, max(self.inf, dim_val)) for dim_val in position]
    
    def plain_init(self):
        return [[random.uniform(self.inf, self.sup) for i in range(self.dim)] for a in range(self.n_agents)]
    
    def fit(self, reset=True):
        self.init_population()
        self.log_func(map(lambda s: s["pos"], self.population), colors=["green", "purple"], legend="PSO - Initialization", padding=1)

        self.best_agent = self.get_best_agent()
        best_cycle = self.best_agent

        for i in tqdm(range(self.n_cycles)):
            self.population = [self.update_agent(e) for e in self.population]
            # velocity calculations and displacement
            self.population = [self.move_agent(e) for e in self.population]
            self.log_func(map(lambda s: s["pos"], self.population), colors=["green", "purple"], legend="PSO - Swarm", it=i+1)
            # Update of the best solution
            best_cycle = self.get_best_agent()
            if self.isbetter_func(best_cycle["bestfit"],self.best_agent["bestfit"]):
                self.best_agent = best_cycle

            self.log_func(self.best_agent['bestpos'], legend="PSO - Best", text=f"{int(self.best_agent['bestfit'])}", colors=["green", "blue"], it=i+1, fitness=self.best_agent['bestfit'])

        self.log_func(self.best_agent['bestpos'], legend="PSO - Best final", text=f"{int(self.best_agent['bestfit'])}", colors=["green", "blue"], it=self.n_cycles, fitness=self.best_agent['bestfit'])
    
    def display_result(self, sols, colors=None, legend="", text="", it=0, fitness=None, **kwargs):
        if legend.split(" - ")[-1].startswith("Best") and it % (self.n_cycles / 5) == 0:
            print("Itération", it)
            print(f"{legend} : {text}")
            print(sols)
            if fitness is not None:
                print("fitness =", fitness)
            print("--------------------\n")

## Initialization

In [3]:
def initSwarm(nb,dim,eval_func,init_function):
    positions = init_function()
    fits = [eval_func(pos) for pos in positions]
    return [{'vit':[0]*dim, 'pos':positions[i], 'fit':fits[i], 'bestpos':positions[i], 'bestfit':fits[i], 'bestvois':[]} for i in range(nb)]

In [4]:
# Return the particle with the best fitness
def maxParticle(p1,p2,isbetter_func):
    if isbetter_func(p1["fit"], p2["fit"]):
        return p1 
    else:
        return p2

# Returns a copy of the particle with the best fitness in the population
def getBest(swarm,isbetter_func):
    return dict(reduce(lambda acc, e: maxParticle(acc,e,isbetter_func),swarm[1:],swarm[0]))

In [5]:
# Update information for the particles of the population (swarm)
def update(particle,bestParticle,isbetter_func):
    nv = dict(particle)
    if isbetter_func(particle["fit"],particle["bestfit"]):
        nv['bestpos'] = particle["pos"][:]
        nv['bestfit'] = particle["fit"]
    nv['bestvois'] = bestParticle["bestpos"][:]
    return nv

# Calculate the velocity and move a particule
def move(particle, dim, eval_function, bounding_function, psi, cmax):
    nv = dict(particle)

    velocity = [0]*dim
    for i in range(dim):
        velocity[i] = (particle["vit"][i]*psi + \
        cmax*random.uniform()*(particle["bestpos"][i] - particle["pos"][i]) + \
        cmax*random.uniform()*(particle["bestvois"][i] - particle["pos"][i]))

    new_pos = [particle["pos"][i] + velocity[i] for i in range(dim)]
    
    start_time = time.time()
    try:
        position = bounding_function(new_pos)   
    except ValueError:
        position = particle["pos"]

    nv['vit'] = velocity
    nv['pos'] = position
    nv['fit'] = eval_function(position)
    return nv

In [6]:
# MAIN LOOP
def fit(eval_function, 
        bounding_function,
        init_function,
        isbetter_function,
        nb_particle, 
        dim, 
        nb_cycles, 
        psi=0.7,
        cmax=1.47,
        log_function=print,
        log_initialization=False):
    Htemps = []       # temps
    Hbest = []        # distance

    # initialization of the population
    swarm = initSwarm(nb_particle,dim,eval_func=eval_function,init_function=init_function)
    if log_initialization:
        log_function(map(lambda s: s["pos"], swarm), colors=["green", "purple"], legend="PSO - Initialization", padding=1)
    # initialization of the best solution
    best = getBest(swarm, isbetter_function)
    best_cycle = best

    for i in tqdm(range(nb_cycles)):
        #Update informations
        swarm = [update(e,best_cycle,isbetter_function) for e in swarm]
        # velocity calculations and displacement
        swarm = [move(particle=e, dim=dim, psi=psi, cmax=cmax, eval_function=eval_function, bounding_function=bounding_function) for e in swarm]
        log_function(map(lambda s: s["pos"], swarm), colors=["green", "purple"], legend="PSO - Swarm", it=i+1)
        # Update of the best solution
        best_cycle = getBest(swarm, isbetter_function)
        if isbetter_function(best_cycle["bestfit"],best["bestfit"]):
            best = best_cycle
            # draw(best['pos'], best['fit'])

        # historization of data
        log_function(best['bestpos'], legend="PSO - Best", text=f"{int(best['bestfit'])}", colors=["green", "blue"], it=i+1, fitness=best['bestfit'])

    #displaying result on the console
    log_function(best['bestpos'], legend="PSO - Best final", text=f"{int(best['bestfit'])}", colors=["green", "blue"], it=nb_cycles, fitness=best['bestfit'])
    return best

## Main test

In [7]:
if __name__ == "__main__":
    
    def sphere(x):
        return reduce(lambda acc,e: acc + e * e, x, 0)
    
    optimizer = PSO(eval_func=sphere, dim=5, inf=-50, sup=50)
    optimizer.fit()

 50%|████▉     | 248/500 [00:00<00:00, 1264.07it/s]

Itération 100
PSO - Best : 0
[-0.04204249991641035, 0.1663901150069171, 0.23570950297982776, -0.2841574083758789, -0.17966997473945703]
fitness = 0.19803894452400705
--------------------

Itération 200
PSO - Best : 0
[0.001915555280648234, -0.005583671452388093, 0.00045763928135621884, -0.013332568004979381, -0.00951455738977354]
fitness = 0.0003033403445639675
--------------------

Itération 300
PSO - Best : 0
[0.00020032672349168433, 9.692624122787199e-06, 0.0004575280737093784, -3.23892236531687e-05, 1.6107191419184553e-05]
fitness = 2.5086518476378297e-07
--------------------



100%|██████████| 500/500 [00:00<00:00, 1263.09it/s]

Itération 400
PSO - Best : 0
[1.067412902331986e-05, 9.09619157007941e-06, -1.8727950730379696e-05, -3.5970027450383847e-06, 1.4328557082291127e-05]
fitness = 7.656598468538814e-10
--------------------

Itération 500
PSO - Best : 0
[1.7936242955824602e-07, 1.2338385842630496e-07, -6.573030417512279e-07, 9.440838981569806e-07, 6.067184801130155e-07]
fitness = 1.7388434672225436e-12
--------------------

Itération 500
PSO - Best final : 0
[1.7936242955824602e-07, 1.2338385842630496e-07, -6.573030417512279e-07, 9.440838981569806e-07, 6.067184801130155e-07]
fitness = 1.7388434672225436e-12
--------------------

